In [1]:
import tensorflow as tf
import pandas as pd
import keras
train = pd.read_csv("dataset_train_set.csv")
test = pd.read_csv("dataset_dev_set.csv")

Using TensorFlow backend.


In [2]:
X_train = train.drop('target',axis=1)
y_train = train['target']
X_test = test.drop('target',axis=1)
y_test = test['target']

In [3]:
X_train = X_train[['aPosLS','hPosLS','hGD5','aSpent','aPtsPG','hSPG','hGCPG','aPos','aSTPG','aGSPG','hSTPG']].copy()
X_test = X_test[['aPosLS','hPosLS','hGD5','aSpent','aPtsPG','hSPG','hGCPG','aPos','aSTPG','aGSPG','hSTPG']].copy()

In [4]:
X_train.head()

,aPosLS,hPosLS,hGD5,aSpent,aPtsPG,hSPG,hGCPG,aPos,aSTPG,aGSPG,hSTPG
0,18,18,0.0,18.07,0.0,0.0,0.0,13.0,0.0,0.0,0.0
1,9,5,0.0,8.92,0.0,0.0,0.0,20.0,0.0,0.0,0.0
2,12,14,0.0,29.52,0.0,0.0,0.0,15.0,0.0,0.0,0.0
3,15,16,0.0,0.00,0.0,0.0,0.0,17.0,0.0,0.0,0.0
4,13,3,0.0,31.03,0.0,0.0,0.0,8.0,0.0,0.0,0.0


In [5]:
print(X_train.shape)
X_train_norm = tf.keras.utils.normalize(X_train.values,axis=1)
X_test_norm = tf.keras.utils.normalize(X_test.values,axis=1)
X_train_norm[1000]

(4180, 11)


array([ 0.23525906,  0.28231087, -0.00941036,  0.19055984,  0.06069684,
        0.58617147,  0.04516974,  0.56462174,  0.24899819,  0.06069684,
        0.30781295])

In [6]:
y_train.value_counts()

2    1971
0    1120
1    1089
Name: target, dtype: int64

In [7]:
whome = 1
wdraw = 1971/1089
waway = 1971/1120
print("{},{},{}".format(whome,wdraw,waway))

1,1.8099173553719008,1.7598214285714286


In [8]:
class_weights = {0:1.7598,
                 1:1.8099,
                 2:1}

In [9]:
model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(12, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(12, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(3, activation = tf.nn.softmax))

In [10]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [11]:
model.fit(X_train_norm, y_train.values, epochs=50, validation_data=(X_test_norm,y_test), class_weight=class_weights)

Train on 4180 samples, validate on 1520 samples
Epoch 1/50
4180/4180 [==============================] - 1s 165us/step - loss: 1.0947 - acc: 0.3514 - val_loss: 1.0829 - val_acc: 0.4039
Epoch 2/50
4180/4180 [==============================] - 0s 87us/step - loss: 1.0745 - acc: 0.4780 - val_loss: 1.0534 - val_acc: 0.5007
Epoch 3/50
4180/4180 [==============================] - 0s 64us/step - loss: 1.0561 - acc: 0.4935 - val_loss: 1.0367 - val_acc: 0.5013
Epoch 4/50
4180/4180 [==============================] - 0s 60us/step - loss: 1.0447 - acc: 0.4792 - val_loss: 1.0172 - val_acc: 0.5039
Epoch 5/50
4180/4180 [==============================] - 0s 65us/step - loss: 1.0385 - acc: 0.4768 - val_loss: 1.0139 - val_acc: 0.5066
Epoch 6/50
4180/4180 [==============================] - 0s 68us/step - loss: 1.0364 - acc: 0.4687 - val_loss: 1.0033 - val_acc: 0.5138
Epoch 7/50
4180/4180 [==============================] - 0s 63us/step - loss: 1.0339 - acc: 0.4617 - val_loss: 1.0068 - val_acc: 0.5099
Epoch 

In [12]:
val_loss, val_acc = model.evaluate(X_test_norm, y_test)
print(val_loss, val_acc)

1520/1520 [==============================] - 0s 52us/step
1.0013098741832533 0.5171052631578947


In [13]:
predictions = model.predict_classes(X_test_norm)
preds = list(predictions)
pd.Series(list(predictions)).value_counts()

2    581
0    554
1    385
dtype: int64

In [14]:
predictions[0]

2

In [15]:
correct_pred_dict = {0:0,1:0,2:0}
for i in range(len(preds)):
    pred = preds[i]
    real = y_test[i]
    if pred == real:
        correct_pred_dict[pred] += 1
correct_pred_dict

{0: 272, 1: 129, 2: 385}

In [16]:
outcomes_incorrectly_caught_dict = {0:0,1:0,2:0}
for i in range(len(preds)):
    pred = preds[i]
    real = y_test[i]
    if pred != real:
        outcomes_incorrectly_caught_dict[real] += 1
outcomes_incorrectly_caught_dict

{0: 188, 1: 243, 2: 303}